# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,8.85,53,75,1.03,CA,1722626889
1,1,blackmans bay,-43.0167,147.3167,8.74,72,100,0.45,AU,1722626820
2,2,pekan,3.4836,103.3996,25.06,91,100,1.49,MY,1722627153
3,3,cockburn town,21.4612,-71.1419,28.36,81,46,9.00,TC,1722627154
4,4,taunsa,30.7048,70.6505,33.88,51,14,2.38,PK,1722627155


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

city_map = city_data_df.hvplot.points("Lng", "Lat", geo=True, size="Humidity", color="City", tiles="OSM", frame_width=700, frame_height=500)

# YOUR CODE HERE


# Display the map
# YOUR CODE HERE
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
nice_weather = city_data_df.loc[(city_data_df["Max Temp"] <50 ) & (city_data_df["Max Temp"] > 10) 
                                & (city_data_df["Cloudiness"] < 60) & (city_data_df["Cloudiness"] > 10) 
                                & (city_data_df["Wind Speed"] < 10) & (city_data_df["Wind Speed"] > 1)]

# Drop any rows with null values
# YOUR CODE HERE
nice_weather = city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
nice_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,8.85,53,75,1.03,CA,1722626889
1,1,blackmans bay,-43.0167,147.3167,8.74,72,100,0.45,AU,1722626820
2,2,pekan,3.4836,103.3996,25.06,91,100,1.49,MY,1722627153
3,3,cockburn town,21.4612,-71.1419,28.36,81,46,9.00,TC,1722627154
4,4,taunsa,30.7048,70.6505,33.88,51,14,2.38,PK,1722627155
...,...,...,...,...,...,...,...,...,...,...
585,585,labytnangi,66.6572,66.4183,16.31,90,96,7.07,RU,1722627829
586,586,mogadishu,2.0371,45.3438,24.73,83,100,7.57,SO,1722627801
587,587,amga,60.8953,131.9608,6.54,85,70,1.01,RU,1722627831
588,588,flying fish cove,-10.4217,105.6791,23.82,88,75,4.12,CX,1722627832


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = nice_weather[['City', 'Country', 'Lat','Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,53,
1,blackmans bay,AU,-43.0167,147.3167,72,
2,pekan,MY,3.4836,103.3996,91,
3,cockburn town,TC,21.4612,-71.1419,81,
4,taunsa,PK,30.7048,70.6505,51,
...,...,...,...,...,...,...
585,labytnangi,RU,66.6572,66.4183,90,
586,mogadishu,SO,2.0371,45.3438,83,
587,amga,RU,60.8953,131.9608,85,
588,flying fish cove,CX,-10.4217,105.6791,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 90000
params = {"categories": "accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address =name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
pekan - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
taunsa - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
bamboo flat - nearest hotel: No hotel found
rockland - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
dodge city - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
nanao - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
hastings - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
madang - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,iqaluit,CA,63.7506,-68.5145,53,No hotel found
1,blackmans bay,AU,-43.0167,147.3167,72,No hotel found
2,pekan,MY,3.4836,103.3996,91,No hotel found
3,cockburn town,TC,21.4612,-71.1419,81,No hotel found
4,taunsa,PK,30.7048,70.6505,51,No hotel found
...,...,...,...,...,...,...
585,labytnangi,RU,66.6572,66.4183,90,No hotel found
586,mogadishu,SO,2.0371,45.3438,83,No hotel found
587,amga,RU,60.8953,131.9608,85,No hotel found
588,flying fish cove,CX,-10.4217,105.6791,88,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE